<a href="https://colab.research.google.com/github/rootAkash/reinforcement_learning/blob/master/ppo_tf.keras/ppo_cnn_battlezone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gym[all]
!pip install box2d-py
!apt-get install python-opengl -y
!apt install xvfb -y

# Special gym environment
#!pip install gym[atari]

     |████████████████████████████████| 122kB 2.7MB/s 
     |████████████████████████████████| 450kB 8.6MB/s 
     |████████████████████████████████| 204kB 13.7MB/s 
  ERROR: Failed building wheel for mujoco-py
  Running setup.py clean for mujoco-py
Failed to build mujoco-py
    Running setup.py install for mujoco-py ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-zevmxqsd/mujoco-py/setup.py'"'"'; __file__='"'"'/tmp/pip-install-zevmxqsd/mujoco-py/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-19b7s44a/install-record.txt --single-version-externally-managed --compile Check the logs for full command output.
  Using cached https://files.pythonhosted.org/packages/06/bd/6cdc3fd994b0649dcf5d9bad85bd9e26172308bbe9a421bfc6fdbf5081a6/bo

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

%cd /gdrive/My Drive/ll_clean

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive/My Drive/ll_clean


In [ ]:
#retro isnt working so atari
import tensorflow as tf
import numpy as np
import gym
import time
import cv2

tf.compat.v1.disable_eager_execution()

env=gym.make("BattleZone-v0")
env=env.unwrapped#removes step restriction

s_dim =(50,50,3) #senv.observation_space.shape[0]
print(s_dim)
a_dim = env.action_space.n
print(a_dim)
#a_bound = env.action_space.high
#print(a_bound)
DUMMY_ACTION, DUMMY_VALUE = np.zeros((1,a_dim)), np.zeros((1, 1))



state_inputs = tf.keras.Input(shape=(50,50,3), name='state')
advantage = tf.keras.Input(shape=(1, ), name="Advantage")
old_prediction = tf.keras.Input(shape=(a_dim,), name="Old_Prediction")

x = tf.keras.layers.Conv2D(filters=64, kernel_size=3,strides=(1,1), padding='valid', activation='relu')(state_inputs)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=3,strides=(1,1), padding='valid', activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=3,strides=(2,2), padding='valid', activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=3,strides=(1,1), padding='valid', activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=3,strides=(2,2), padding='valid', activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=3,strides=(1,1), padding='valid', activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=3,strides=(1,1), padding='valid', activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=3,strides=(1,1), padding='valid', activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=3,strides=(1,1), padding='valid', activation='relu')(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(64*2, activation='relu')(x)
action_outputs = tf.keras.layers.Dense(a_dim, activation='softmax')(x)


def proximal_policy_optimization_loss(advantage, old_prediction):
	loss_clipping = 0.2
	entropy_loss = 0.0
	#y_true = one hot actions , y_pred = prob output
	def loss(y_true, y_pred):
		prob = y_true * y_pred
		old_prob = y_true * old_prediction
		r = prob / (old_prob + 1e-10)
		loss = -tf.keras.backend.mean(tf.keras.backend.minimum(r * advantage, tf.keras.backend.clip(r, min_value=1 - loss_clipping,max_value=1 + loss_clipping) * advantage) + entropy_loss * (prob * tf.keras.backend.log(prob + 1e-10)))
		return loss
	return loss	
policy= tf.keras.Model(inputs=[state_inputs, advantage, old_prediction], outputs=[action_outputs], name='p_actor_model')
policy.compile(loss=proximal_policy_optimization_loss(advantage=advantage,old_prediction=old_prediction), optimizer=tf.keras.optimizers.Adam(lr=0.00001))# custom lAdam(lr=0.0001) to be defined
policy.summary()
x = tf.keras.layers.Conv2D(filters=64, kernel_size=3,strides=(1,1), padding='valid', activation='relu')(state_inputs)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=3,strides=(1,1), padding='valid', activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=3,strides=(2,2), padding='valid', activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=3,strides=(1,1), padding='valid', activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=3,strides=(2,2), padding='valid', activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=3,strides=(1,1), padding='valid', activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=3,strides=(1,1), padding='valid', activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=3,strides=(1,1), padding='valid', activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=3,strides=(1,1), padding='valid', activation='relu')(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(64*2, activation='relu')(x)
value_outputs = tf.keras.layers.Dense(1, activation=None)(x)
critic= tf.keras.Model(inputs=state_inputs, outputs=value_outputs, name='p_critic_model')
critic.compile(loss="mse", optimizer=tf.keras.optimizers.Adam(lr=0.001), metrics=['accuracy'])
critic.summary()


def max(a,b):
	if a>b:
		return a
	else:
		return b
def abs(a):
	if a>=0:
		return a
	else:
		return -a


class Memory:
	def __init__(self):
		self.batch_s = []
		self.batch_a = []
		self.batch_r = []
		self.batch_s_ = []
		self.batch_done = []
		self.batch_pred =[]

	def store(self, s, a, s_, r, done,pred):
		self.batch_s.append(s)
		self.batch_a.append(a)
		self.batch_r.append(r)
		self.batch_s_.append(s_)
		self.batch_done.append(done)
		self.batch_pred.append(pred)    

	def clear(self):
		self.batch_s.clear()
		self.batch_a.clear()
		self.batch_r.clear()
		self.batch_s_.clear()
		self.batch_done.clear()
		self.batch_pred.clear()    

	def cnt_samples(self):
		return len(self.batch_s)
def onehot(a,s):
	i = np.zeros(a_dim)
	i[a]=1
	return i
def save_weights():
	actorpath="/gdrive/My Drive/ll_clean/ppo_cnn_battle_zone_A.h5"
	criticpath="/gdrive/My Drive/ll_clean/ppo_cnn_battle_zone_C.h5"
	policy.save_weights(actorpath)
	critic.save_weights(criticpath)
	print("saved")
def load_weights():
	actorpath="/gdrive/My Drive/ll_clean/ppo_cnn_battle_zone_A.h5"
	criticpath="/gdrive/My Drive/ll_clean/ppo_cnn_battle_zone_C.h5"
	policy.load_weights(actorpath)
	critic.load_weights(criticpath)
	print("loaded")

def gae_calc(val,val_,rew,done):
	mask=1 
	gae=0
	gamma=0.99
	lambd = 0.95
	returns=np.zeros_like(val)
	for i in reversed(range(0,len(val))):
		mask=1
		if done[i]:
			mask = 0 	
		delta=rew[i]+gamma*val_[i]*mask - val[i]
		gae=delta+gamma*lambd*mask*gae
		returns[i]=gae+val[i]
	return returns
#############################################################################################################	
def preprocess(state):
	state= state[55:155,:,:]
	state = cv2.resize( state, (50,50) )
	return state/225.0

load_weights()

episodes = 3000
steps = 5000
memory=Memory()
render=0
s=env.reset()
avg = []
t = time.time()
for episode in range(1,episodes):
	
	stp = 0 
	done =False
	s=env.reset()
	s = preprocess(s)
	rew = 0 	
	if episode>2000:
		render=0
		
	while not done:
		if render:
			env.render()
			#time.sleep(0.01)
		pred_action = policy.predict([np.array([s]),DUMMY_VALUE,DUMMY_ACTION])# prob distribution
		action = np.random.choice(np.arange(pred_action.shape[1]), p=pred_action.ravel())# action chosen
		#random_action= np.random.choice(np.arange(pred_action.shape[1]))
		#if steps% 2 == 0 and episode<100:
		#	action = random_action
		action_one_hot=onehot(action,a_dim)# acton matrix
		s_, reward, done, info = env.step(action)

		s_ = preprocess(s_)	
		memory.store(s,action_one_hot.ravel(),s_,reward,done,pred_action.ravel())# s, a, s_, r, done ,pred
		if stp> steps:
			done =True 
		s=s_
		rew+=reward
		stp+=1
	# updation
	avg.append(rew)
	obs =np.array( memory.batch_s)
	values = critic.predict(np.array(memory.batch_s))
	values_ = critic.predict(np.array(memory.batch_s_))
	returns = gae_calc(values,values_,memory.batch_r,memory.batch_done)	
	advantage=returns-values
	old_Prediction=memory.batch_pred
	old_Prediction=np.array(old_Prediction)
	action=np.array(memory.batch_a)########################
	#print(".")
	#print(episode)
	policy.fit(x=[obs,advantage, old_Prediction],y=action,batch_size=64,shuffle=True, epochs=10, verbose=False)
	critic.fit([obs],[returns], batch_size=64, shuffle=True, epochs=10, verbose=False)
	#print(actor_loss,critic_loss)
	memory.clear()
	if episode % 100== 0:
		print("current ",episode , " average of last 100 episodes :", np.mean(np.array(avg)), " max:", np.max(np.array(avg)), " min:", np.min(np.array(avg)) )
		avg.clear()
		print(" total time taken (min): " ,(time.time()-t)/60)
		save_weights()
		t = time.time()
  
	

(50, 50, 3)
18
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "p_actor_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
state (InputLayer)              [(None, 50, 50, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 48, 48, 64)   1792        state[0][0]                      
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 46, 46, 64)   36928       conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 22, 22, 64)   36928       conv2d_1[0][0]          